**Общая информация**

**Срок сдачи:** 21 октября 2019, 08:30 

**Штраф за опоздание:** по 0.5 балла за 24 часа задержки. Через 10 дней домашнее задание сгорает.

При отправлении ДЗ указывайте фамилию в названии файла
Присылать ДЗ необходимо в виде ссылки на свой github репозиторий на почту ml1.sphere@mail.ru с указанием темы в следующем формате:

[ML0919, Задание 1] Фамилия Имя.

Используйте данный Ipython Notebook при оформлении домашнего задания.


**Штрафные баллы:**

1. Невыполнение PEP8 -0.5 баллов
2. Отсутствие фамилии в имени скрипта (скрипт должен называться по аналогии со stroykova_hw1.ipynb) -0.5 баллов
3. Все строчки должны быть выполнены. Нужно, чтобы output команды можно было увидеть уже в git'е. В противном случае -0.5 баллов


In [2]:
import copy
import itertools
import re
import numpy as np
from scipy.sparse import csr_matrix
from scipy.stats import  mode
import matplotlib.pyplot as plt

import seaborn as sns
from sklearn import datasets
from sklearn.base import ClassifierMixin
from sklearn.datasets import fetch_mldata, fetch_20newsgroups

from sklearn.neighbors.base import NeighborsBase, KNeighborsMixin, SupervisedIntegerMixin 
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier, KDTree

from sklearn.model_selection import cross_validate, KFold

from sklearn.metrics.pairwise import euclidean_distances

%load_ext pycodestyle_magic

##### Задание 1 (1 балл)
Реализовать KNN в классе MyKNeighborsClassifier (обязательное условие: точность не ниже sklearn реализации)
Разберитесь самостоятельно, какая мера расстояния используется в KNeighborsClassifier дефолтно и реализуйте свой алгоритм именно с этой мерой. Самостоятельно разберитесь, как считается score из KNeighborsClassifier и реализуйте аналог в своём классе. Score не должен уступать значению KNN из sklearn

In [88]:
# %%pycodestyle


class MyKNeighborsClassifier(NeighborsBase, KNeighborsMixin,
                             SupervisedIntegerMixin, ClassifierMixin):

    def __init__(self, n_neighbors, algorithm='brute'):

        self.n_neighbors = n_neighbors
        self.algorithm = algorithm

    def fit(self, X, y):

        if self.n_neighbors < X.shape[0]:
            self.X_train = X
            self.y_train = np.array(y)
            self.classes = np.unique(y)

        else:
            print("Parameter n_neingbors is higher than the \
                    number of observations")
            return

    def predict(self, X):

        n_neighbors = self.n_neighbors
        algorithm = self.algorithm

        if n_neighbors < X.shape[0]:
            if algorithm == 'brute':
                neigh_ind, neigh_dist = self.kneighbors_brute(X)
            elif algorithm == 'kd_tree':
                tree = KDTree(self.X_train, leaf_size=2)
                neigh_dist, neigh_ind = tree.query(X, k=n_neighbors)
            else:
                print("The name of algorithm is incorrect. \
                Please, use 'brute' or 'kd_tree' ")
            y_pred = np.transpose(mode(self.y_train[neigh_ind], axis=1)[0])[0]
            return y_pred
        else:
            print("The number of observations in \
                  test set is higher than n_neingbors")

    def kneighbors_brute(self, X):

        n_neighbors = self.n_neighbors

        distances = euclidean_distances(X, self.X_train)
        # indices of closes n_neighbors elements
        sort_indices = distances.argsort(axis=1)[:, :n_neighbors]
        # sorted distances
        distances.sort()
        distances[:, :n_neighbors]

        return sort_indices, distances[:, :n_neighbors]

    def predict_proba(self, X):

        n_neighbors = self.n_neighbors
        classes = self.classes
        algorithm = self.algorithm

        if n_neighbors < X.shape[0]:

            if algorithm == 'brute':
                neigh_ind, neigh_dist = self.kneighbors_brute(X)
            elif algorithm == 'kd_tree':
                tree = KDTree(self.X_train, leaf_size=2)
                neigh_dist, neigh_ind = tree.query(X, k=n_neighbors)
            else:
                print("The name of algorithm is incorrect. \
                Please, use 'brute' or 'kd_tree' ")

            y_pred = self.y_train[neigh_ind]

            probabilities = np.count_nonzero(y_pred == classes[0], axis=1)

            for i in range(1, len(classes)):
                probabilities = np.vstack(
                    (probabilities, np.count_nonzero(
                        y_pred == classes[i], axis=1)))

            probabilities = probabilities/probabilities.sum(axis=0)

            return np.transpose(probabilities)

        else:
            print("The number of observations in \
                  test set is higher than n_neingbors")

    def score(self, X, y):

        y_pred = self.predict(X)

        differing_labels = np.count_nonzero(y_pred - y)

        return 1 - differing_labels / y_pred.shape[0]

**IRIS**

В библиотеке scikit-learn есть несколько датасетов из коробки. Один из них [Ирисы Фишера](https://ru.wikipedia.org/wiki/%D0%98%D1%80%D0%B8%D1%81%D1%8B_%D0%A4%D0%B8%D1%88%D0%B5%D1%80%D0%B0)

In [4]:
iris = datasets.load_iris()

In [5]:
X_train, X_test, y_train, y_test = train_test_split(
    iris.data, iris.target, test_size=0.1, stratify=iris.target)

In [72]:
clf = KNeighborsClassifier(n_neighbors=2, algorithm='brute')
my_clf = MyKNeighborsClassifier(n_neighbors=2, algorithm='brute')

In [61]:
clf.fit(X_train, y_train)
my_clf.fit(X_train, y_train)

In [12]:
assert abs(my_clf.score(X_test, y_test) - clf.score(X_test, y_test))<0.005, "Score must be simillar"

**Задание 2 (0.5 балла)**

Давайте попробуем добиться скорости работы на fit, predict и predict_proba сравнимой со sklearn для iris.
Для этого используем numpy

In [14]:
%time clf.fit(X_train, y_train)

CPU times: user 1.32 ms, sys: 0 ns, total: 1.32 ms
Wall time: 1.13 ms


KNeighborsClassifier(algorithm='brute', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=2, p=2,
                     weights='uniform')

In [15]:
%time my_clf.fit(X_train, y_train)

CPU times: user 340 µs, sys: 0 ns, total: 340 µs
Wall time: 295 µs


In [16]:
%time clf.predict(X_test)

CPU times: user 5.18 ms, sys: 237 µs, total: 5.41 ms
Wall time: 4.32 ms


array([1, 2, 1, 0, 0, 0, 1, 2, 2, 1, 1, 2, 0, 1, 0])

In [17]:
%time my_clf.predict(X_test)

CPU times: user 1.19 ms, sys: 3.43 ms, total: 4.62 ms
Wall time: 3.45 ms


array([1, 2, 1, 0, 0, 0, 1, 2, 2, 1, 1, 2, 0, 1, 0])

In [18]:
%time clf.predict_proba(X_test)

CPU times: user 2.24 ms, sys: 439 µs, total: 2.68 ms
Wall time: 2.04 ms


array([[0. , 1. , 0. ],
       [0. , 0. , 1. ],
       [0. , 1. , 0. ],
       [1. , 0. , 0. ],
       [1. , 0. , 0. ],
       [1. , 0. , 0. ],
       [0. , 1. , 0. ],
       [0. , 0. , 1. ],
       [0. , 0. , 1. ],
       [0. , 1. , 0. ],
       [0. , 0.5, 0.5],
       [0. , 0. , 1. ],
       [1. , 0. , 0. ],
       [0. , 1. , 0. ],
       [1. , 0. , 0. ]])

In [19]:
%time my_clf.predict_proba(X_test)

CPU times: user 1.96 ms, sys: 385 µs, total: 2.34 ms
Wall time: 2.64 ms


array([[0. , 1. , 0. ],
       [0. , 0. , 1. ],
       [0. , 1. , 0. ],
       [1. , 0. , 0. ],
       [1. , 0. , 0. ],
       [1. , 0. , 0. ],
       [0. , 1. , 0. ],
       [0. , 0. , 1. ],
       [0. , 0. , 1. ],
       [0. , 1. , 0. ],
       [0. , 0.5, 0.5],
       [0. , 0. , 1. ],
       [1. , 0. , 0. ],
       [0. , 1. , 0. ],
       [1. , 0. , 0. ]])

###### Задание 3 (1 балл)
Добавьте algorithm='kd_tree' в реализацию KNN (использовать KDTree из sklearn.neighbors). Необходимо добиться скорости работы на fit,  predict и predict_proba сравнимой со sklearn для iris.
Для этого используем numpy. Score не должен уступать значению KNN из sklearn

In [20]:
clf = KNeighborsClassifier(n_neighbors=2, algorithm='kd_tree')
my_clf = MyKNeighborsClassifier(n_neighbors=2, algorithm='kd_tree')

In [21]:
tree = KDTree(X_train, leaf_size=2)

In [22]:
neigh_dist, neigh_ind = tree.query(X_test, k=2) 

In [23]:
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.1, stratify=iris.target)

In [24]:
%time clf.fit(X_train, y_train)

CPU times: user 0 ns, sys: 3.01 ms, total: 3.01 ms
Wall time: 2.4 ms


KNeighborsClassifier(algorithm='kd_tree', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=2, p=2,
                     weights='uniform')

In [25]:
%time my_clf.fit(X_train, y_train)

CPU times: user 421 µs, sys: 83 µs, total: 504 µs
Wall time: 395 µs


In [26]:
%time clf.predict(X_test)

CPU times: user 1.46 ms, sys: 3.48 ms, total: 4.94 ms
Wall time: 3.75 ms


array([1, 1, 2, 1, 2, 0, 2, 0, 0, 1, 0, 0, 1, 2, 2])

In [27]:
%time my_clf.predict(X_test)

CPU times: user 4.73 ms, sys: 935 µs, total: 5.67 ms
Wall time: 5.91 ms


array([1, 1, 2, 1, 2, 0, 2, 0, 0, 1, 0, 0, 1, 2, 2])

In [28]:
%time clf.predict_proba(X_test)

CPU times: user 2.82 ms, sys: 0 ns, total: 2.82 ms
Wall time: 2.38 ms


array([[0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.]])

In [29]:
%time my_clf.predict_proba(X_test)

CPU times: user 3.3 ms, sys: 0 ns, total: 3.3 ms
Wall time: 2.44 ms


array([[0., 1., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.],
       [1., 0., 0.],
       [0., 1., 0.],
       [0., 0., 1.],
       [0., 0., 1.]])

In [30]:
assert abs(my_clf.score(X_test, y_test) - clf.score(X_test,y_test))<0.005, "Score must be simillar"

**Задание 4 (2.5 балла)**

Рассмотрим новый датасет 20 newsgroups

In [31]:
newsgroups = fetch_20newsgroups(subset='train',remove=['headers','footers', 'quotes'])

In [32]:
data = newsgroups['data']
target = newsgroups['target']

Переведите во всех документах все буквы в нижний регистр и замените во всех документах символы, не
являющиеся буквами и цифрами, на пробелы. Далее разбейте текста по пробельным символам на токены(термы/слова). Удалите текста, содержащие только пробелы.

In [33]:
# %%pycodestyle
def tokenize(file_str):
    '''
    file_str - str
    '''
    file_str = file_str.lower()
    file_str = re.sub(r"[^a-zA-Z0-9]", " ", file_str)

    file_l = file_str.split()

    return file_l

In [34]:
# %%pycodestyle
data_tok1 = [tokenize(l) for l in copy.deepcopy(data)]

data_tok = []
target_new = []

for line, y in zip(data_tok1, target):
    if line != []:
        data_tok.append(line)
        target_new.append(y)

In [35]:
len(data_tok)

11004

In [36]:
assert all(isinstance(row, (list, tuple)) for row in data_tok), "please convert each line into a list of tokens (strings)"
assert all(all(isinstance(tok, str) for tok in row) for row in data_tok), "please convert each line into a list of tokens (strings)"
is_latin = lambda tok: all('a' <= x.lower() <= 'z' for x in tok)
assert all(map(lambda l: not is_latin(l) or l.islower() , map(' '.join, data_tok))), "please make sure that you lowercase the data and drop spaced texts"

Преобразуйте датасет в разреженную матрицу scipy.sparse.csr_matrix, где значение x в позиции (i, j)
означает, что в документе i слово j встретилось x раз

In [37]:
indptr = [0]
indices = []
data = []
vocabulary = {}

for d in data_tok:
    for term in d:
        if (len(term) < 3) or (len(term) > 20) or not term.isdigit():
            continue
        else:
            index = vocabulary.setdefault(term, len(vocabulary))
            indices.append(index)
            data.append(1)
    indptr.append(len(indices))

word_count_m = csr_matrix((data, indices, indptr), dtype=int).toarray()

In [38]:
word_count_m.shape

(11004, 4393)

In [39]:
condition1 = word_count_m.sum(axis=0) < 100
# condition1 = condition1[np.newaxis, :]

condition2 = word_count_m.sum(axis=0) > 2
# condition2 = condition2[np.newaxis, :]
np.count_nonzero(condition1), np.count_nonzero(condition2)

(4369, 1463)

In [40]:
word_count_m = word_count_m[:,np.logical_or(condition1, condition2)]

In [44]:
word_count_m.shape

(11004, 4393)

*Так мы получили векторное представление наших текстов. Значит можно приступать к задаче обучения модели*

Реализуйте разбиение выборки для кросс-валидации на 3 фолдах. Разрешено использовать sklearn.cross_validation

In [89]:
my_clf = MyKNeighborsClassifier(n_neighbors=2, algorithm='brute')

In [90]:
my_clf.fit(word_count_m[:-500], target_new[:-500])

In [91]:
my_clf.X_train.shape, my_clf.y_train.shape

((10504, 4393), (10504,))

In [ ]:
my_clf.predict(word_count_m[:-2]).shape

In [55]:
word_count_m

array([[0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 1, 2, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 1]])

In [162]:
scores = cross_validate(my_clf, word_count_m, target_new, cv=3)

IndexError: index 468 is out of bounds for axis 0 with size 135

In [ ]:
np.mean(scores)

Напишите метод, позволяющий найти оптимальное количество ближайших соседей(дающее максимальный score в среднем на валидации на 3 фолдах).
Постройте график зависимости среднего score от количества соседей. Можно рассмотреть число соседей от 1 до 10.

Как изменится качество на валидации, если:

1. Используется косинусная метрика вместо евклидовой.
2. К текстам применяется TfIdf преобразование( sklearn.feature_extraction.text.TfidfTransformer)

Сравните модели, выберите лучшую.

Загрузим  теперь test  часть нашей выборки и преобразуем её аналогично с train частью. Не забудьте, что наборы слов в train и test части могут отличаться.

In [ ]:
newsgroups = fetch_20newsgroups(subset='test',remove=['headers','footers', 'quotes'])

Оценим качество(score) вашей лучшей модели на test части датасета. Отличается ли оно от кросс-валидации? Попробуйте сделать выводы, почему отличается качество.